<a href="https://colab.research.google.com/github/MBarak9/Operations-Research-with-python/blob/main/Planification_multi_p%C3%A9riode.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from pulp import *
import pandas as pd

In [ ]:
prob = LpProblem("Synchronisation_de_la_Chaine_logistique",LpMinimize)

In [ ]:
mois = ['Janvier','Février','Mars','Avril','Mai','Juin']

achat_cost = [1000,975,1000,980,1020,1025]
achat_cost_dict = dict(zip(mois,achat_cost))

prod_cost = [160,150,150,160,175,165]
prod_cost_dict = dict(zip(mois,prod_cost))

prod_capacity = [6,5,4,4,4,3]
prod_capacity_dict = dict(zip(mois,prod_capacity))

achat_min = [4,3,5,2,4,5]
achat_min_dict = dict(zip(mois,achat_min))

achat_max = [6,4,7,3,7,6]
achat_max_dict = dict(zip(mois,achat_max))

stock = {'entr_i':2,'entr_f':2,'entr_max':3,'entr_cost':20,'mag_max':1,'mag_cost':25,'mag_i':0,'mag_f':0}

demand = 4

In [ ]:
#Les variables de décisions
A_dict = LpVariable.dicts('Achat',mois,0,None,LpInteger)
E_dict = LpVariable.dicts('Entreposer',mois,0,None,LpInteger)
P_dict = LpVariable.dicts('Produire',mois,0,None,LpInteger)
M_dict = LpVariable.dicts('Stocker',mois,0,None,LpInteger)

In [ ]:
#La fonction-objectif
prob += lpSum([achat_cost_dict[m]*A_dict[m]
               +stock['entr_cost']*E_dict[m]
               +prod_cost_dict[m]*P_dict[m]
               +stock['mag_cost']*M_dict[m] for m in mois])

In [ ]:
#Affichage de la fonction-objectif
prob.objective

980*Achat_Avril + 975*Achat_Février + 1000*Achat_Janvier + 1025*Achat_Juin + 1020*Achat_Mai + 1000*Achat_Mars + 20*Entreposer_Avril + 20*Entreposer_Février + 20*Entreposer_Janvier + 20*Entreposer_Juin + 20*Entreposer_Mai + 20*Entreposer_Mars + 160*Produire_Avril + 150*Produire_Février + 160*Produire_Janvier + 165*Produire_Juin + 175*Produire_Mai + 150*Produire_Mars + 25*Stocker_Avril + 25*Stocker_Février + 25*Stocker_Janvier + 25*Stocker_Juin + 25*Stocker_Mai + 25*Stocker_Mars + 0

In [ ]:
# Les contraintes de capacité
for m in mois:
    prob += A_dict[m] <= achat_max_dict[m]    # Sur les achats
    prob += A_dict[m] >= achat_min_dict[m]     # Sur les achats
    prob += E_dict[m] <= stock['entr_max']      # Sur l'entrepôt
    prob += P_dict[m] <= prod_capacity_dict[m]   # Sur la production
    prob += M_dict[m] <= stock['mag_max']        #Sur le magazin

# Les contraintes de synchronisation
prob += E_dict['Janvier'] == stock['entr_i'] + A_dict['Janvier'] - P_dict['Janvier']
prob += M_dict[mois[0]] == stock['mag_i'] + P_dict[mois[0]] - demand
for i_m in range(1,len(mois)) :
    prob += E_dict[mois[i_m]] == E_dict[mois[i_m-1]] + A_dict[mois[i_m]] - P_dict[mois[i_m]]
    prob += M_dict[mois[i_m]] == M_dict[mois[i_m-1]] + P_dict[mois[i_m]] - demand
prob += E_dict['Juin'] == stock['entr_f']
prob += M_dict[mois[5]] == stock['mag_f']

In [ ]:
prob.variables

<bound method LpProblem.variables of Synchronisation_de_la_Chaine_logistique:
MINIMIZE
980*Achat_Avril + 975*Achat_Février + 1000*Achat_Janvier + 1025*Achat_Juin + 1020*Achat_Mai + 1000*Achat_Mars + 20*Entreposer_Avril + 20*Entreposer_Février + 20*Entreposer_Janvier + 20*Entreposer_Juin + 20*Entreposer_Mai + 20*Entreposer_Mars + 160*Produire_Avril + 150*Produire_Février + 160*Produire_Janvier + 165*Produire_Juin + 175*Produire_Mai + 150*Produire_Mars + 25*Stocker_Avril + 25*Stocker_Février + 25*Stocker_Janvier + 25*Stocker_Juin + 25*Stocker_Mai + 25*Stocker_Mars + 0
SUBJECT TO
_C1: Achat_Janvier <= 6

_C2: Achat_Janvier >= 4

_C3: Entreposer_Janvier <= 3

_C4: Produire_Janvier <= 6

_C5: Stocker_Janvier <= 1

_C6: Achat_Février <= 4

_C7: Achat_Février >= 3

_C8: Entreposer_Février <= 3

_C9: Produire_Février <= 5

_C10: Stocker_Février <= 1

_C11: Achat_Mars <= 7

_C12: Achat_Mars >= 5

_C13: Entreposer_Mars <= 3

_C14: Produire_Mars <= 4

_C15: Stocker_Mars <= 1

_C16: Achat_Avril <=

In [ ]:
prob.solve()
print('Status : ',LpStatus[prob.status])

Status :  Optimal


In [ ]:
print(value(prob.objective))

28095.0


In [ ]:
colonnes = ['Fournisseur','Entrepôt','Production','Magasin']
result_df = pd.DataFrame(columns=colonnes).reindex(index=mois)
result_df['Fournisseur'] = [int(value(A_dict[m])) for m in mois]
result_df['Entrepôt'] = [int(value(E_dict[m])) for m in mois]
result_df['Production'] = [int(value(P_dict[m])) for m in mois]
result_df['Magasin'] = [int(value(M_dict[m])) for m in mois]

In [ ]:
result_df

,Fournisseur,Entrepôt,Production,Magasin
Janvier,4,2,4,0
Février,3,0,5,1
Mars,5,1,4,1
Avril,3,0,4,1
Mai,4,0,4,1
Juin,5,2,3,0


In [ ]:
# affichage des coûts duals et des écarts
o = [{'contraintes':nom, 'coût dual':c.pi, 'écart': c.slack}
for nom, c in prob.constraints.items()]
sens_df = pd.DataFrame(o)
sens_df

,contraintes,coût dual,écart
0,_C1,0.0,2.0
1,_C2,0.0,-0.0
2,_C3,0.0,1.0
3,_C4,0.0,2.0
4,_C5,0.0,1.0
5,_C6,0.0,1.0
6,_C7,0.0,-0.0
7,_C8,0.0,3.0
8,_C9,0.0,-0.0
9,_C10,0.0,-0.0
